In [13]:
# Import the neccessary libraries
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine, text  # Add the 'text' import

# PostgreSQL database connection parameters
db_params = {
    'user': 'yourusername',
    'password': 'yourpassword',
    'host': 'yourhost',
    'port': 'yourport',
    'database': 'yourdbname'
}

# PostgreSQL connection string
connection_str = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["database"]}'

base_url = 'https://euenergy.live/'
start_date = datetime.today()
end_date = datetime.today()
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

list_page_links = []
all_data = []  # List to store dictionaries containing data for each date

for date in date_range:
    formatted_date = date.strftime('%Y-%m-%d')
    link = f'{base_url}?date={formatted_date}'
    list_page_links.append(link)

    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find_all('div', {'id': 'right'})

    countries = [country.get_text() for country in results[0].find_all('a', {'class': 'country_link'})]
    prices = [float(price.get_text().replace(',', '')) for price in results[0].find_all('td', {'class': 'price'})]

    # Create a list of dictionaries for each date
    data_for_date = [{'date': formatted_date, 'country': country, 'price': price} for country, price in zip(countries, prices)]
    all_data.extend(data_for_date)

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(all_data)

# Group by 'Date' and 'Country', and calculate the average of 'Price' to avoid duplicates 
df = df.groupby(['date', 'country'])["price"].mean().reset_index()

# # transform the values to positive
# df = df['price'] = abs(df["price"])

# Create a SQLAlchemy engine to connect to the PostgreSQL database
engine = create_engine(connection_str)

# Create the table in the PostgreSQL database
with engine.connect() as connection:
    sql_statement = '''
        CREATE TABLE IF NOT EXISTS energy_data (
            id SERIAL PRIMARY KEY,
            date DATE,
            country VARCHAR(255),
            price NUMERIC,
            CONSTRAINT unique_date_country UNIQUE (date, country)
        )
    '''
    connection.execute(text(sql_statement))

# Send the DataFrame to the PostgreSQL database
df.to_sql('energy_data', engine, if_exists='append', index=False, method='multi')

27

In [2]:
df

,date,country,price
0,2024-01-26,Austria,75.00
1,2024-01-26,Belgium,67.65
2,2024-01-26,Bulgaria,92.32
3,2024-01-26,Switzerland,78.11
4,2024-01-26,Czechia,76.90
5,2024-01-26,Germany,59.23
6,2024-01-26,Denmark,55.80
7,2024-01-26,Denmark,50.43
8,2024-01-26,Estonia,106.58
9,2024-01-26,Spain,72.56


In [3]:
df['date'] = pd.to_datetime(df['date'])

In [5]:
df['price'] = df['price'].astype(float)

In [6]:
df = df.groupby(['date', 'country'])["price"].mean().reset_index()

In [7]:
df

,date,country,price
0,2024-01-26,Austria,75.0000
1,2024-01-26,Belgium,67.6500
2,2024-01-26,Bulgaria,92.3200
3,2024-01-26,Croatia,83.5200
4,2024-01-26,Czechia,76.9000
5,2024-01-26,Denmark,53.1150
6,2024-01-26,Estonia,106.5800
7,2024-01-26,Finland,78.8400
8,2024-01-26,France,65.8200
9,2024-01-26,Germany,59.2300
